In [ ]:
# 思维链（Chain of Thought Reasoning）通常是一次性给出所有提示或问题，并要求模型分步推理。
# 链式推理（Chain of Thought）则强调将问题拆分成多个部分，分阶段进行推理

In [ ]:
# 在上一章中，了解了如何评估 LLM 模型在 有明确正确答案 的情况下的输出，我们可以编写一个函数来判断 LLM 输出是否正确地分类并列出产品。
# 然而，如果 LLM 用于生成文本，而不仅仅是分类问题的答案呢？接下来，我们将探讨如何评估这种类型的 LLM 输出的方法。

In [1]:
# 导入 OpenAI API
import os
import openai
import sys
# sys.path.append('../..')
import utils_en
import utils_zh

In [2]:
from openai import OpenAI
client = OpenAI()
def get_completion_from_messages(messages, 
                                 model="gpt-4o-mini", 
                                 temperature=0.5, 
                                 max_tokens=500):
    '''
    封装一个支持更多参数的自定义访问 gpt-4o-mini 的函数
    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-4o-mini(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0.5，增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message.content

In [21]:
# 运行问答系统获得一个复杂回答
# 用户消息
customer_msg = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs or TV related products do you have?"""
# 从问题中抽取商品名
products_by_category = utils_en.get_products_from_query(customer_msg)
# 将商品名转化为列表
category_and_product_list = utils_en.read_string_to_list(products_by_category)
# 查找商品对应的信息
product_info = utils_en.get_mentioned_product_info(category_and_product_list)
# 由信息生成回答
assistant_answer = utils_en.answer_user_msg(user_msg=customer_msg, product_info=product_info)

In [ ]:
# 所谓链式是指在整个处理过程中会多次调用模型生成回复，把事情分步处理
# 所谓思维链是指让模型处理问题时分步骤进行推理,但是整个处理过程还是一次调用

In [3]:
'''
注意：限于模型对中文理解能力较弱，中文 Prompt 可能会随机出现不成功，可以多次运行；也非常欢迎同学探究更稳定的中文 Prompt
'''
# 用户消息
customer_msg = f"""
告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。
另外，你们这有什么 TVs ？"""
# 第一次调用模型从问题中抽取商品和对应的类别名
products_by_category = utils_zh.get_products_from_query(customer_msg)
# 将商品和对应的类别名转化为列表形式
category_and_product_list = utils_zh.read_string_to_list(products_by_category)
# 查找商品对应的具体信息
product_info = utils_zh.get_mentioned_product_info(category_and_product_list)
# 由用户消息和上面提取的商品信息,再次调用助手产生人性化的回复
assistant_answer = utils_zh.answer_user_msg(user_msg=customer_msg, product_info=product_info)
print(assistant_answer) 

关于您提到的产品：

1. **SmartX ProPhone**
   - **显示**: 6.1英寸
   - **存储**: 128GB
   - **相机**: 12MP双摄
   - **网络**: 5G
   - **价格**: 899.99元
   - **描述**: 一款功能强大的智能手机，配备先进的相机功能。
   - **评分**: 4.6/5

2. **FotoSnap DSLR Camera**
   - **传感器**: 24.2MP
   - **视频**: 1080p
   - **显示屏**: 3英寸LCD
   - **镜头**: 可更换镜头
   - **价格**: 599.99元
   - **描述**: 捕捉惊人的照片和视频，这款多功能单反相机是您的理想选择。
   - **评分**: 4.7/5

关于电视方面，我们有几款不错的选择：

1. **CineView 4K TV**
   - **尺寸**: 55英寸
   - **分辨率**: 4K
   - **价格**: 599.99元
   - **评分**: 4.8/5

2. **CineView 8K TV**
   - **尺寸**: 65英寸
   - **分辨率**: 8K
   - **价格**: 2999.99元
   - **评分**: 4.9/5

3. **CineView OLED TV**
   - **尺寸**: 55英寸
   - **分辨率**: 4K
   - **价格**: 1499.99元
   - **评分**: 4.7/5

您对哪款产品最感兴趣呢？或者您有其他问题吗？


In [ ]:
# 使用 GPT 评估回答是否正确
# 我们希望您能从中学到一个设计模式，即当您可以指定一个评估 LLM 输出的标准列表时，您实际上可以使用另一个 API 调用来评估您的第一个 LLM 输出

In [4]:
display(customer_msg, product_info)

'\n告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？'

[{'name': 'SmartX ProPhone',
  'category': 'Smartphones and Accessories',
  'brand': 'SmartX',
  'model_number': 'SX-PP10',
  'warranty': '1 year',
  'rating': 4.6,
  'features': ['6.1-inch display', '128GB storage', '12MP dual camera', '5G'],
  'description': 'A powerful smartphone with advanced camera features.',
  'price': 899.99},
 {'name': 'FotoSnap DSLR Camera',
  'category': 'Cameras and Camcorders',
  'brand': 'FotoSnap',
  'model_number': 'FS-DSLR200',
  'warranty': '1 year',
  'rating': 4.7,
  'features': ['24.2MP sensor',
   '1080p video',
   '3-inch LCD',
   'Interchangeable lenses'],
  'description': 'Capture stunning photos and videos with this versatile DSLR camera.',
  'price': 599.99},
 {'name': 'CineView 4K TV',
  'category': 'Televisions and Home Theater Systems',
  'brand': 'CineView',
  'model_number': 'CV-4K55',
  'warranty': '2 years',
  'rating': 4.8,
  'features': ['55-inch display', '4K resolution', 'HDR', 'Smart TV'],
  'description': 'A stunning 4K TV with v

In [22]:
# 问题、上下文
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': product_info
}

In [23]:
def eval_with_rubric(test_set, assistant_answer):
    """
    使用 GPT API 评估生成的回答

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """

    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    # 要求 GPT 作为一个助手评估回答正确性
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by looking at the context that the customer service \
    agent is using to generate its response. 
    """

    # 具体指令
    user_message = f"""\
You are evaluating a submitted answer to a question based on the context \
that the agent uses to answer the question.
Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Context]: {context}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the context. \
Ignore any differences in style, grammar, or punctuation.
Answer the following questions:
    - Is the Assistant response based only on the context provided? (Y or N)
    - Does the answer include information that is not provided in the context? (Y or N)
    - Is there any disagreement between the response and the context? (Y or N)
    - Count how many questions the user asked. (output a number)
    - For each question that the user asked, is there a corresponding answer to it?
      Question 1: (Y or N)
      Question 2: (Y or N)
      ...
      Question N: (Y or N)
    - Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [7]:
def eval_with_rubric(test_set, assistant_answer):
    """
    使用 GPT API 评估生成的回答

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    
    cust_msg = test_set['customer_msg'] # 用户输入的查询
    context = test_set['context'] # 上下文(就是根据用户查询调用模型生成的相关问题的商品信息)
    completion = assistant_answer # 模型根据用户查询和调用模型查找到的信息,生成的友好回复
    # 人设
    system_message = """\
    你是一位助理，通过查看客户服务代理使用的上下文来评估客户服务代理回答用户问题的情况。
    """
    # 具体指令
    user_message = f"""\
    你正在根据代理使用的上下文评估对问题的提交答案。以下是数据：
    [开始]
    ************
    [用户问题]: {cust_msg}
    ************
    [使用的上下文]: {context}
    ************
    [客户代理的回答]: {completion}
    ************
    [结束]

    请将提交的答案的事实内容与上下文进行比较，忽略样式、语法或标点符号上的差异。
    回答以下问题：
    助手的回应是否只基于所提供的上下文？（是或否）
    回答中是否包含上下文中未提供的信息？（是或否）
    回应与上下文之间是否存在任何不一致之处？（是或否）
    计算用户提出了多少个问题。（输出一个数字）
    对于用户提出的每个问题，是否有相应的回答？
    问题1：（是或否）
    问题2：（是或否）
    ...
    问题N：（是或否）
    在提出的问题数量中，有多少个问题在回答中得到了回应？（输出一个数字）
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [ ]:
# 技术型模型用于基础信息检索。
# 社交型模型用于处理人性化和友好的服务回答。
# 检验型模型用于确保回答的质量和准确性。
# 这种分工明确的模型设计，能确保在提供服务时既有准确的技术支持，也能确保用户体验，同时又通过评估机制进行验证，确保服务质量。

In [ ]:
# 到目前细分的话一共调用了三个模型,第一个是根据用户输入查询相关商品粗略信息的技术型模型
# 第二个是根据用户查询和查询到的商品信息生成友好服务的社交型模型
# 第三个是根据用户查询和技术模型提供的商品信息和社交模型的回复对社交模型的回复进行评估的检验型模型

In [24]:
print(customer_msg)


tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs or TV related products do you have?


In [25]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

- Is the Assistant response based only on the context provided? (Y)
- Does the answer include information that is not provided in the context? (N)
- Is there any disagreement between the response and the context? (N)
- Count how many questions the user asked. (2)
- For each question that the user asked, is there a corresponding answer to it?
  Question 1: (Y)
  Question 2: (Y)
- Of the number of questions asked, how many of these questions were addressed by the answer? (2)


In [ ]:
# 给出一个标准回答，要求其评估生成回答与标准回答的差距
# 在经典的自然语言处理技术中，有一些传统的度量标准用于衡量 LLM 输出与人类专家编写的输出的相似度。例如，BLUE 分数可用于衡量两段文本的相似程度。
# 实际上有一种更好的方法，即使用 Prompt。您可以指定 Prompt，使用 Prompt 来比较由 LLM 自动生成的客户服务代理响应与人工理想响应的匹配程度。

In [26]:
# 用户问题和标准答案的字典
test_set_ideal = {
    'customer_msg': """\
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs or TV related products do you have?""",
    'ideal_answer':"""\
Of course!  The SmartX ProPhone is a powerful \
smartphone with advanced camera features. \
For instance, it has a 12MP dual camera. \
Other features include 5G wireless and 128GB storage. \
It also has a 6.1-inch display.  The price is $899.99.

The FotoSnap DSLR Camera is great for \
capturing stunning photos and videos. \
Some features include 1080p video, \
3-inch LCD, a 24.2MP sensor, \
and interchangeable lenses. \
The price is 599.99.

For TVs and TV related products, we offer 3 TVs \


All TVs offer HDR and Smart TV.

The CineView 4K TV has vibrant colors and smart features. \
Some of these features include a 55-inch display, \
'4K resolution. It's priced at 599.

The CineView 8K TV is a stunning 8K TV. \
Some features include a 65-inch display and \
8K resolution.  It's priced at 2999.99

The CineView OLED TV lets you experience vibrant colors. \
Some features include a 55-inch display and 4K resolution. \
It's priced at 1499.99.

We also offer 2 home theater products, both which include bluetooth.\
The SoundMax Home Theater is a powerful home theater system for \
an immmersive audio experience.
Its features include 5.1 channel, 1000W output, and wireless subwoofer.
It's priced at 399.99.

The SoundMax Soundbar is a sleek and powerful soundbar.
It's features include 2.1 channel, 300W output, and wireless subwoofer.
It's priced at 199.99

Are there any questions additional you may have about these products \
that you mentioned here?
Or may do you have other questions I can help you with?
    """
}

In [12]:
'''基于中文Prompt的验证集'''
test_set_ideal = {
    'customer_msg': """\
告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？""",
    'ideal_answer':"""\
SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。
FotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。
我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年
    """
}

In [14]:
print(test_set_ideal)

{'customer_msg': '告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？', 'ideal_answer': 'SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。\nFotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。\n我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年\n    '}


In [27]:
def eval_vs_ideal(test_set, assistant_answer):
    """
    评估回复是否与理想答案匹配

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by comparing the response to the ideal (expert) response
    Output a single letter and nothing else. 
    """

    user_message = f"""\
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Expert]: {ideal}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
    (A) The submitted answer is a subset of the expert answer and is fully consistent with it.
    (B) The submitted answer is a superset of the expert answer and is fully consistent with it.
    (C) The submitted answer contains all the same details as the expert answer.
    (D) There is a disagreement between the submitted answer and the expert answer.
    (E) The answers differ, but these differences don't matter from the perspective of factuality.
  choice_strings: ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [ ]:
# 子集与超集的关系：如果A是B的子集，那么B就是A的超集，反之亦然。
# 超集与子集的区别：超集的元素多于子集，或者二者完全相同（即相等）。子集只能包含超集中的一部分或全部元素。
# 对应到你的问题：
# (A) 提交的答案是专家答案的子集，并且与之完全一致：提交的答案包含专家答案中的所有细节，但没有额外的信息，且没有差异。
# (B) 提交的答案是专家答案的超集，并且与之完全一致：提交的答案包含专家答案的所有细节，并且提供了额外的信息，且没有差异。
# (C) 提交的答案包含与专家答案完全相同的细节：提交的答案和专家答案内容完全相同，没有遗漏或额外信息。
# (D) 提交的答案与专家答案存在分歧：提交的答案与专家答案存在不一致的地方，可能是事实错误或观点冲突。
# (E) 答案存在差异，但从事实的角度来看这些差异并不重要：提交的答案与专家答案有细微差别，但这些差异不会影响答案的实质性准确性。

In [15]:
def eval_vs_ideal(test_set, assistant_answer):
    """
    评估回复是否与理想答案匹配
    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    cust_msg = test_set['customer_msg'] # 用户问题
    ideal = test_set['ideal_answer'] # 标准答案
    completion = assistant_answer # 助手回复
    # 设定背景
    system_message = """\
    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。
    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 
    """
    # 设定用户消息:里面有问题,标准答案,助手回复的答案
    user_message = f"""\
    您正在比较一个给定问题的提交答案和专家答案。数据如下:
    [开始]
    ************
    [问题]: {cust_msg}
    ************
    [专家答案]: {ideal}
    ************
    [提交答案]: {completion}
    ************
    [结束]

    比较提交答案的事实内容与专家答案。忽略样式、语法或标点符号上的差异。
    提交的答案可能是专家答案的子集、超集，或者与之冲突。确定适用的情况，并通过选择以下选项之一回答问题：
    （A）提交的答案是专家答案的子集，并且与之完全一致。
    （B）提交的答案是专家答案的超集，并且与之完全一致。
    （C）提交的答案包含与专家答案完全相同的细节。
    （D）提交的答案与专家答案存在分歧。
    （E）答案存在差异，但从事实的角度来看这些差异并不重要。
    选项：ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [ ]:
# 这个评分标准来自于 OpenAI 开源评估框架，这是一个非常棒的框架，其中包含了许多评估方法，既有 OpenAI 开发人员的贡献，
# 也有更广泛的开源社区的贡献。
# 在这个评分标准中，我们要求 LLM 针对提交答案与专家答案进行信息内容的比较，并忽略其风格、语法和标点符号等方面的差异，
# 但关键是我们要求它进行比较，并输出从A到E的分数，具体取决于提交的答案是否是专家答案的子集、超集或完全一致，这可能意
# 味着它虚构或编造了一些额外的事实。
# LLM 将选择其中最合适的描述。

In [28]:
print(assistant_answer)

The **SmartX ProPhone** is a powerful smartphone featuring a 6.1-inch display, 128GB of storage, and a 12MP dual camera. It supports 5G connectivity, making it a great choice for fast internet access. It’s priced at $899.99 and comes with a 1-year warranty.

The **FotoSnap DSLR Camera** is designed for photography enthusiasts, offering a 24.2MP sensor, 1080p video recording, and a 3-inch LCD screen. It supports interchangeable lenses, allowing for versatility in your photography. This camera is available for $599.99, also with a 1-year warranty.

As for TVs and related products, we have a variety of options:

1. **CineView 4K TV** - 55-inch, 4K resolution, Smart TV - $599.99
2. **CineView 8K TV** - 65-inch, 8K resolution, Smart TV - $2999.99
3. **CineView OLED TV** - 55-inch, 4K resolution, Smart TV - $1499.99
4. **SoundMax Home Theater** - 5.1 channel, 1000W output - $399.99
5. **SoundMax Soundbar** - 2.1 channel, 300W output - $199.99

Is there a specific feature or size you’re looki

In [17]:
eval_vs_ideal(test_set_ideal, assistant_answer)
# 对于该生成回答，GPT 判断生成内容是标准回答的一个子集

'C'

In [18]:
assistant_answer_2 = "life is like a box of chocolates"
eval_vs_ideal(test_set_ideal, assistant_answer_2)
# 对于明显异常答案，GPT 判断为不一致

'D'

In [19]:
print(assistant_answer)

关于您提到的产品：

1. **SmartX ProPhone**
   - **显示**: 6.1英寸
   - **存储**: 128GB
   - **相机**: 12MP双摄
   - **网络**: 5G
   - **价格**: 899.99元
   - **描述**: 一款功能强大的智能手机，配备先进的相机功能。
   - **评分**: 4.6/5

2. **FotoSnap DSLR Camera**
   - **传感器**: 24.2MP
   - **视频**: 1080p
   - **显示屏**: 3英寸LCD
   - **镜头**: 可更换镜头
   - **价格**: 599.99元
   - **描述**: 捕捉惊人的照片和视频，这款多功能单反相机是您的理想选择。
   - **评分**: 4.7/5

关于电视方面，我们有几款不错的选择：

1. **CineView 4K TV**
   - **尺寸**: 55英寸
   - **分辨率**: 4K
   - **价格**: 599.99元
   - **评分**: 4.8/5

2. **CineView 8K TV**
   - **尺寸**: 65英寸
   - **分辨率**: 8K
   - **价格**: 2999.99元
   - **评分**: 4.9/5

3. **CineView OLED TV**
   - **尺寸**: 55英寸
   - **分辨率**: 4K
   - **价格**: 1499.99元
   - **评分**: 4.7/5

您对哪款产品最感兴趣呢？或者您有其他问题吗？


In [29]:
eval_vs_ideal(test_set_ideal, assistant_answer)

'C'

In [ ]:
# 希望您从本章中学到两个设计模式。
# 即使没有专家提供的理想答案，只要能制定一个评估标准，就可以使用一个 LLM 来评估另一个 LLM 的输出。
# 如果您可以提供一个专家提供的理想答案，那么可以帮助您的 LLM 更好地比较特定助手输出是否与专家提供的理想答案相似。
# 希望这可以帮助您评估 LLM 系统的输出，以便在开发期间持续监测系统的性能，并使用这些工具不断评估和改进系统的性能。